In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip3 install catboost
!pip3 install category_encoders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 205.2 kB/s eta 0:00:00


In [ ]:
!pip install -U scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 20.7 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2


In [ ]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.2/404.2 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.3/225.3 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 7.2 MB/s eta 0:00:00


In [ ]:
import unicodedata
from pathlib import Path

import catboost as cb
import optuna
import lightgbm as lgb
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from category_encoders import CountEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import TargetEncoder
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.preprocessing import RobustScaler
import datetime

In [ ]:

exp_name = 'exp00025'

outputs_path = f'/content/drive/MyDrive/Colab Notebooks/signate2023/{exp_name}/'

submission_dir = outputs_path + 'submissions/'
submission_path = submission_dir + f'submission_{exp_name}.csv'

model_dir = outputs_path + f'signate-models/'

for dir in [model_dir]:
    os.makedirs(dir, exist_ok=True)

In [ ]:
INPUT_DIR = Path("/content/drive/MyDrive/Colab Notebooks/signate2023/")

In [ ]:
train_df = pd.read_csv(INPUT_DIR / "train.csv")
test_df = pd.read_csv(INPUT_DIR / "test.csv")
sub_df = pd.read_csv(INPUT_DIR / "submit_sample.csv", names=["id", "price"])

In [ ]:
# yearが3000年以降のものはおかしいので2000年に置換
err_idx = train_df.query("year >= 3000").index
train_df.loc[err_idx, "year"] = train_df.loc[err_idx, "year"].apply(lambda x: x - 1000)

# yearが3000年以降のものはおかしいので2000年に置換
err_idx = test_df.query("year >= 3000").index
test_df.loc[err_idx, "year"] = test_df.loc[err_idx, "year"].apply(lambda x: x - 1000)

In [ ]:
# yaerの外れ値除去 (2023年より未来ののyearの値を持つtrainのレコードを削除する)
train_df = train_df[train_df.year <= datetime.datetime.now().year].copy()
train_df = train_df.reset_index(drop=True)

In [ ]:
train_df

,id,region,year,manufacturer,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color,state,price
0,0,nashville,1949,bmw,excellent,6 cylinders,gas,115148,clean,manual,rwd,mid-size,convertible,orange,NaN,27587
1,1,state college,2013,toyota,fair,8 cylinders,gas,172038,clean,automatic,rwd,full-size,sedan,silver,pa,4724
2,2,wichita,1998,ford,good,6 cylinders,gas,152492,clean,automatic,fwd,full-size,SUV,silver,ks,10931
3,3,albany,2014,ford,excellent,4 cylinders,gas,104118,clean,manual,fwd,mid-size,SUV,blue,ny,16553
4,4,redding,2005,ford,excellent,6 cylinders,gas,144554,clean,manual,fwd,mid-size,sedan,red,ca,5158
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27526,27527,williamsport,2008,ford,good,6 cylinders,gas,26660,clean,automatic,rwd,compact,truck,black,pa,32212
27527,27528,tulsa,2007,ford,excellent,8 cylinders,gas,108072,clean,automatic,rwd,full-size,pickup,black,NaN,5400
27528,27529,rochester,2019,jeep,like new,6 cylinders,gas,139908,clean,automatic,4wd,mid-size,SUV,white,ny,22227
27529,27530,rochester,2007,jeep,excellent,6 cylinders,gas,112326,clean,automatic,4wd,mid-size,sedan,white,ny,3054


In [ ]:
import unicodedata

# Define a function to normalize manufacturer names
def normalize_manufacturer_name(name):
    # Convert full-width characters to half-width
    name = unicodedata.normalize('NFKC', name)
    # Convert to lowercase
    name = name.lower()
    # Remove non-ASCII characters
    name = ''.join(char for char in name if ord(char) < 128)
    return name


class PreProcessTransformer(TransformerMixin, BaseEstimator):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        # cylinderから数値を取り出す
        X["cylinders"] = X["cylinders"].astype(str).str.extract("(\d+)").astype("float32")

        # sizeの表記揺れを修正
        X["size"] = X["size"].str.replace("ー", "-").astype(str)
        X["size"] = X["size"].str.replace("−", "-").astype(str)

        # Apply the function to the 'manufacturer' column
        X['manufacturer'] = X['manufacturer'].apply(normalize_manufacturer_name)

        X.loc[X['odometer'] == -1, 'odometer'] = np.nan
        X.loc[X['odometer'] == -131869, 'odometer'] = np.nan


        # Define a dictionary for manufacturer name mapping with special characters
        manufacturer_name_mapping = {
            'nisan': 'nissan',
            'toyot': 'toyota',
            'nisѕan': 'nissan',
            'subαru': 'subaru',
            'subru': 'subaru',
            'sαturn': 'saturn',
            'lexu':'lexus',
            'аcura': 'acura',
            'vоlkswagen': 'volkswagen',
            'vlkswagen': 'volkswagen',
            'lexuѕ': 'lexus',
            'ᴄhrysler': 'chrysler',
            'sturn': 'saturn'
        }

        # Replace the manufacturer names using the mapping
        X['manufacturer'] = X['manufacturer'].replace(manufacturer_name_mapping)

        # 走行距離 / 製造年
        X["odometer_per_year"] = X["odometer"] / (2023 - X["year"])

        return X


class RankTransformer(TransformerMixin, BaseEstimator):
    """keyの中でvalueが何番目のものか"""

    def __init__(self, key: str, value: str):
        self.key = key
        self.value = value

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_new = X.copy()
        X_new[self.key] = X_new.groupby(self.key)[self.value].rank(method="dense")
        return X_new[self.get_feature_names_out()]

    def get_feature_names_out(self, input_features=None):
        return [self.key]


class OriginalTransformer(TransformerMixin, BaseEstimator):
    """数値特徴はそのまま、カテゴリ特徴はcategory型に変換"""

    def __init__(self, numeric_cols, categorical_cols):
        self.numeric_cols = numeric_cols
        self.categorical_cols = categorical_cols

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_new = X.copy()

        # 数値変数
        X_new[self.numeric_cols] = X_new[self.numeric_cols].astype("float32")

        # カテゴリ変数
        #X_new[self.categorical_cols] = X_new[self.categorical_cols].astype("category")

        return X_new[self.get_feature_names_out()]

    def get_feature_names_out(self):
        return self.numeric_cols + self.categorical_cols


class CountTransformer(TransformerMixin, BaseEstimator):
    """CountEncoder"""

    def __init__(self):
        pass

    def fit(self, X, y=None):
        self.ce = CountEncoder(cols=X.columns.tolist(), handle_unknown=0)
        self.ce.fit(X)
        return self

    def transform(self, X):
        return self.ce.transform(X)

    def get_feature_names_out(self, input_features=None):
        return input_features


class AggTransformer(TransformerMixin, BaseEstimator):
    """集約特徴量"""

    def __init__(self, key, numeric_cols, agg_func: dict):
        self.key = key
        self.numeric_cols = numeric_cols
        self.agg_func = agg_func

    def fit(self, X, y=None):
        X = X.copy()
        X[self.key] = X[self.key].astype("category")
        self.agg_df = X.groupby(self.key)[self.numeric_cols].agg(self.agg_func)
        self.agg_df.columns = [f"{col}_{func}" for col, func in self.agg_df.columns.tolist()]

        return self

    def transform(self, X):
        X_new = pd.merge(X, self.agg_df, on=self.key, how="left")
        return X_new[self.get_feature_names_out()]

    def get_feature_names_out(self, input_features=None):
        return self.agg_df.columns.tolist()

In [ ]:
numeric_cols = ["year", "odometer", "odometer_per_year"]
categorical_cols = [
    'region',
    "cylinders",
    "manufacturer",
    "condition",
    "fuel",
    "title_status",
    "transmission",
    "drive",
    "size",
    "type",
    "paint_color",
    "state",
]


ct = ColumnTransformer(
    transformers=[
        (
            "ori",
            OriginalTransformer(numeric_cols, categorical_cols),
            categorical_cols + numeric_cols,
        ),
        (
            "ct",
            CountTransformer(),
            categorical_cols
        ),
       # *[
       #      (f"agg_{key}", AggTransformer(key, ["odometer"], {"mean", "std"}), [key] + ["odometer"])
       #      for key in [
       #          "manufacturer",
       #          "cylinders",
       #          "condition",
       #          "fuel",
       #          "drive",
       #          "size",
       #          "type",
       #      ]
       #  ],
    ],
    verbose=True,
)
ct.set_output(transform="pandas")
pipe = Pipeline(
    steps=[
        ("preprocess", PreProcessTransformer()),
        ("ct", ct),
    ]
)

train_feat_df = pipe.fit_transform(train_df, train_df["price"])
test_feat_df = pipe.transform(test_df)

[ColumnTransformer] ........... (1 of 2) Processing ori, total=   0.0s
[ColumnTransformer] ............ (2 of 2) Processing ct, total=   2.1s


In [ ]:
train_feat_df

,ori__year,ori__odometer,ori__odometer_per_year,ori__region,ori__cylinders,ori__manufacturer,ori__condition,ori__fuel,ori__title_status,ori__transmission,...,ct__manufacturer,ct__condition,ct__fuel,ct__title_status,ct__transmission,ct__drive,ct__size,ct__type,ct__paint_color,ct__state
0,1949.0,115148.0,1556.054077,nashville,6.0,bmw,excellent,gas,clean,manual,...,3044,15219,23545,22364,6155,4833,9411,418,144,3303
1,2013.0,172038.0,17203.800781,state college,8.0,toyota,fair,gas,clean,automatic,...,1784,2404,23545,22364,20497,4833,14635,9259,4300,983
2,1998.0,152492.0,6099.680176,wichita,6.0,ford,good,gas,clean,automatic,...,6942,6008,23545,22364,20497,14602,14635,7310,4300,252
3,2014.0,104118.0,11568.666992,albany,4.0,ford,excellent,gas,clean,manual,...,6942,15219,23545,22364,6155,14602,9411,7310,3765,1841
4,2005.0,144554.0,8030.777832,redding,6.0,ford,excellent,gas,clean,manual,...,6942,15219,23545,22364,6155,14602,9411,9259,1418,2702
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27526,2008.0,26660.0,1777.333374,williamsport,6.0,ford,good,gas,clean,automatic,...,6942,6008,23545,22364,20497,4833,3352,3837,10143,983
27527,2007.0,108072.0,6754.500000,tulsa,8.0,ford,excellent,gas,clean,automatic,...,6942,15219,23545,22364,20497,4833,14635,2118,10143,3303
27528,2019.0,139908.0,34977.000000,rochester,6.0,jeep,like new,gas,clean,automatic,...,1297,3810,23545,22364,20497,8096,9411,7310,3624,1841
27529,2007.0,112326.0,7020.375000,rochester,6.0,jeep,excellent,gas,clean,automatic,...,1297,15219,23545,22364,20497,8096,9411,9259,3624,1841


In [ ]:
train_df['price']

0        27587
1         4724
2        10931
3        16553
4         5158
         ...  
27526    32212
27527     5400
27528    22227
27529     3054
27530    20801
Name: price, Length: 27531, dtype: int64

In [ ]:
from scipy.stats import mode
import numpy as np

# モデルIDの作成
train_feat_df['model_id'] = train_feat_df[['ori__manufacturer', 'ori__cylinders', 'ori__fuel', 'ori__transmission', 'ori__drive', 'ori__size', 'ori__type']].apply(lambda x: '_'.join(x.dropna().astype(str)), axis=1)
test_feat_df['model_id'] = test_feat_df[['ori__manufacturer', 'ori__cylinders', 'ori__fuel', 'ori__transmission', 'ori__drive', 'ori__size', 'ori__type']].apply(lambda x: '_'.join(x.dropna().astype(str)), axis=1)

train_df['model_id'] = train_feat_df['model_id']

# 基準価格、基準年度、基準走行距離を格納するための辞書
standard_values = {
    'standard_price': {},
    'standard_year': {},
    'standard_odometer': {}
}

for model_id in train_feat_df['model_id'].unique():
    model_data_features = train_feat_df[train_feat_df['model_id'] == model_id]
    model_data_prices = train_df['price'][model_data_features.index]

    # 価格と特徴量を結合
    model_data = model_data_features.copy()
    model_data['price'] = model_data_prices

    # 価格の最頻値
    price_mode_result, price_count_result = mode(model_data_prices, keepdims=False)
    if price_count_result > 1:
        standard_price = price_mode_result
    else:
        standard_price = model_data_prices.median()

    # 製造年の最頻値
    year_mode_result, year_count_result = mode(model_data_features['ori__year'], keepdims=False)
    if year_count_result > 1:
        standard_year = year_mode_result
    else:
        standard_year = model_data_features['ori__year'].median()

    # 走行距離の最頻値
    odometer_data = model_data_features['ori__odometer'].dropna()
    if odometer_data.size == 0:
        print(f"Inserting NaN for model_id: {model_id} as it has no odometer data.")
        standard_odometer = np.nan # ここで NaN を挿入
    else:
        odometer_mode_result, odometer_count_result = mode(odometer_data, keepdims=False)
        if odometer_count_result > 1:
            standard_odometer = odometer_mode_result
        else:
            standard_odometer = odometer_data.median()


    # 結果を辞書に格納
    standard_values['standard_price'][model_id] = standard_price
    standard_values['standard_year'][model_id] = standard_year
    standard_values['standard_odometer'][model_id] = standard_odometer

# 最初の5つのモデルの基準値を表示して確認
for key, values in standard_values.items():
    print(f"{key}: {list(values.items())[:5]}")


Inserting NaN for model_id: mercedes-benz_6.0_automatic_rwd_full-size as it has no odometer data.
Inserting NaN for model_id: mazda_4.0_gas_other_fwd_compact_hatchback as it has no odometer data.
Inserting NaN for model_id: nissan_6.0_gas_automatic_fwd_compact_wagon as it has no odometer data.
Inserting NaN for model_id: volvo_8.0_gas_automatic_4wd_mid-size_SUV as it has no odometer data.
Inserting NaN for model_id: hyundai_4.0_hybrid_manual_fwd_full-size_sedan as it has no odometer data.
Inserting NaN for model_id: honda_6.0_gas_other_rwd_full-size_SUV as it has no odometer data.
Inserting NaN for model_id: jeep_6.0_gas_automatic_4wd_sub-compact_SUV as it has no odometer data.
Inserting NaN for model_id: chevrolet_6.0_gas_manual_rwd_sub-compact_coupe as it has no odometer data.
Inserting NaN for model_id: mini_4.0_gas_manual_rwd_compact_hatchback as it has no odometer data.
Inserting NaN for model_id: chevrolet_8.0_gas_other_fwd_full-size_van as it has no odometer data.
Inserting NaN 

In [ ]:
# 基準値を適用する関数
def apply_standard_values(df, y, standard_values):
    df = pd.concat([df,y],axis=1)
    df['standard_price'] = df['model_id'].map(standard_values['standard_price'])
    df['standard_year'] = df['model_id'].map(standard_values['standard_year'])
    df['standard_odometer'] = df['model_id'].map(standard_values['standard_odometer'])

    # 価格と基準価格との比率
    df['price_ratio'] = df['price'] / df['standard_price']

    # 年数と基準年数との差
    df['year_diff'] = df['ori__year'] - df['standard_year']

    # 走行距離と基準走行距離との差
    df['odometer_diff'] = df['ori__odometer'] - df['standard_odometer']
    df.drop(['price'], axis=1, inplace=True)

    return df

# 訓練データに基準値を適用
train_feat_df = apply_standard_values(train_feat_df,train_df['price'], standard_values)

# 年ごとの平均走行距離
mean_odometer_per_year = train_feat_df.groupby('ori__year')['ori__odometer'].mean()

# 年ごとの平均走行距離と実際の走行距離との差
train_feat_df['odometer_year_diff'] = train_feat_df['ori__odometer'] - train_feat_df['ori__year'].map(mean_odometer_per_year)

# テストデータにも同様の処理を適用（価格に関する処理は除く）
test_feat_df['standard_year'] = test_feat_df['model_id'].map(standard_values['standard_year'])
test_feat_df['standard_odometer'] = test_feat_df['model_id'].map(standard_values['standard_odometer'])
test_feat_df['year_diff'] = test_feat_df['ori__year'] - test_feat_df['standard_year']
test_feat_df['odometer_diff'] = test_feat_df['ori__odometer'] - test_feat_df['standard_odometer']
test_feat_df['odometer_year_diff'] = test_feat_df['ori__odometer'] - test_feat_df['ori__year'].map(mean_odometer_per_year)

# 結果の確認
train_feat_df[['price_ratio', 'year_diff', 'odometer_diff', 'odometer_year_diff']].head()


,price_ratio,year_diff,odometer_diff,odometer_year_diff
0,1.774426,-54.0,5465.0,67966.000000
1,1.010265,4.0,14152.0,63772.390625
2,1.401949,-15.0,23557.0,9714.390625
3,1.897409,4.0,-31232.0,-842.046875
4,0.738492,-8.0,0.0,-5064.781250


In [ ]:
test_feat_df

,ori__year,ori__odometer,ori__odometer_per_year,ori__region,ori__cylinders,ori__manufacturer,ori__condition,ori__fuel,ori__title_status,ori__transmission,...,ct__size,ct__type,ct__paint_color,ct__state,model_id,standard_year,standard_odometer,year_diff,odometer_diff,odometer_year_diff
0,2015.0,92553.0,11569.125000,western slope,4.0,chevrolet,excellent,gas,clean,automatic,...,14635,7310,1418,3303,chevrolet_4.0_gas_automatic_fwd_full-size_SUV,2012.0,105178.0,3.0,-12625.0,-6103.218750
1,2013.0,134385.0,13438.500000,roseburg,4.0,nissan,like new,gas,salvage,automatic,...,9411,9259,10143,577,nissan_4.0_gas_automatic_fwd_mid-size_sedan,2014.0,107921.0,-1.0,26464.0,26119.390625
2,2011.0,102489.0,8540.750000,akron / canton,4.0,volkswagen,good,gas,clean,automatic,...,14635,9259,10143,849,volkswagen_4.0_gas_automatic_fwd_full-size_sedan,2013.0,115055.0,-2.0,-12566.0,-12658.710938
3,2016.0,64310.0,9187.142578,denver,6.0,jeep,excellent,diesel,clean,automatic,...,9411,7310,1418,705,jeep_6.0_diesel_automatic_4wd_mid-size_SUV,2013.0,114543.0,3.0,-50233.0,-20847.601562
4,1999.0,180839.0,7534.958496,hickory / lenoir,8.0,honda,excellent,gas,rebuilt,automatic,...,9411,7310,4300,636,honda_8.0_gas_automatic_4wd_mid-size_SUV,NaN,NaN,NaN,NaN,30437.593750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27532,2016.0,90902.0,12986.000000,great falls,6.0,gmc,excellent,gas,rebuilt,automatic,...,14635,3837,10143,249,gmc_6.0_gas_automatic_fwd_full-size_truck,2011.0,173029.0,5.0,-82127.0,5744.398438
27533,2012.0,27234.0,2475.818115,las vegas,4.0,nissan,excellent,gas,rebuilt,automatic,...,9411,9259,3624,369,nissan_4.0_gas_automatic_fwd_mid-size_sedan,2014.0,107921.0,-2.0,-80687.0,-84914.156250
27534,2002.0,99761.0,4750.523926,phoenix,6.0,bmw,excellent,gas,clean,automatic,...,14635,2255,3765,706,bmw_6.0_gas_automatic_rwd_full-size_coupe,2010.0,85348.0,-8.0,14413.0,-47190.359375
27535,2006.0,162279.0,9545.823242,colorado springs,6.0,bmw,excellent,gas,clean,automatic,...,9411,9259,4300,705,bmw_6.0_gas_automatic_4wd_mid-size_sedan,2013.0,107378.0,-7.0,54901.0,16982.500000


In [ ]:
train_feat_df

,ori__year,ori__odometer,ori__odometer_per_year,ori__region,ori__cylinders,ori__manufacturer,ori__condition,ori__fuel,ori__title_status,ori__transmission,...,ct__paint_color,ct__state,model_id,standard_price,standard_year,standard_odometer,price_ratio,year_diff,odometer_diff,odometer_year_diff
0,1949.0,115148.0,1556.054077,nashville,6.0,bmw,excellent,gas,clean,manual,...,144,3303,bmw_6.0_gas_manual_rwd_mid-size_convertible,15547.0,2003.0,109683.0,1.774426,-54.0,5465.0,67966.000000
1,2013.0,172038.0,17203.800781,state college,8.0,toyota,fair,gas,clean,automatic,...,4300,983,toyota_8.0_gas_automatic_rwd_full-size_sedan,4676.0,2009.0,157886.0,1.010265,4.0,14152.0,63772.390625
2,1998.0,152492.0,6099.680176,wichita,6.0,ford,good,gas,clean,automatic,...,4300,252,ford_6.0_gas_automatic_fwd_full-size_SUV,7797.0,2013.0,128935.0,1.401949,-15.0,23557.0,9714.390625
3,2014.0,104118.0,11568.666992,albany,4.0,ford,excellent,gas,clean,manual,...,3765,1841,ford_4.0_gas_manual_fwd_mid-size_SUV,8724.0,2010.0,135350.0,1.897409,4.0,-31232.0,-842.046875
4,2005.0,144554.0,8030.777832,redding,6.0,ford,excellent,gas,clean,manual,...,1418,2702,ford_6.0_gas_manual_fwd_mid-size_sedan,6984.5,2013.0,144554.0,0.738492,-8.0,0.0,-5064.781250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27526,2008.0,26660.0,1777.333374,williamsport,6.0,ford,good,gas,clean,automatic,...,10143,983,ford_6.0_gas_automatic_rwd_compact_truck,32212.0,2008.0,26660.0,1.000000,0.0,0.0,-113939.484375
27527,2007.0,108072.0,6754.500000,tulsa,8.0,ford,excellent,gas,clean,automatic,...,10143,3303,ford_8.0_gas_automatic_rwd_full-size_pickup,8857.0,2007.0,118969.0,0.609687,0.0,-10897.0,-33087.140625
27528,2019.0,139908.0,34977.000000,rochester,6.0,jeep,like new,gas,clean,automatic,...,3624,1841,jeep_6.0_gas_automatic_4wd_mid-size_SUV,12527.0,2011.0,118883.0,1.774327,8.0,21025.0,71956.851562
27529,2007.0,112326.0,7020.375000,rochester,6.0,jeep,excellent,gas,clean,automatic,...,3624,1841,jeep_6.0_gas_automatic_4wd_mid-size_sedan,4200.5,2013.0,135676.5,0.727056,-6.0,-23350.5,-28833.140625


In [ ]:
state_dic = {}

# データ数的に、重複は気にせず上書きしていっても大した時間にはならない
for i in train_feat_df.index:
    # ただし、欠損値で上書きされると困るので、そこだけケア
    if pd.isna(train_feat_df.at[i, 'ori__state'])==False:
        state_dic[train_feat_df.at[i, 'ori__region']] = train_feat_df.at[i, 'ori__state']

train_feat_df.loc[pd.isna(train_feat_df['ori__state']).index, 'ori__state'] = train_feat_df.loc[pd.isna(train_feat_df['ori__state']).index, 'ori__region'].map(state_dic)
test_feat_df.loc[pd.isna(test_feat_df['ori__state']).index, 'ori__state'] = test_feat_df.loc[pd.isna(test_feat_df['ori__state']).index, 'ori__region'].map(state_dic)

In [ ]:
pd.set_option('display.max_columns', None)
train_feat_df

,ori__year,ori__odometer,ori__odometer_per_year,ori__region,ori__cylinders,ori__manufacturer,ori__condition,ori__fuel,ori__title_status,ori__transmission,ori__drive,ori__size,ori__type,ori__paint_color,ori__state,ct__region,ct__cylinders,ct__manufacturer,ct__condition,ct__fuel,ct__title_status,ct__transmission,ct__drive,ct__size,ct__type,ct__paint_color,ct__state,model_id,standard_price,standard_year,standard_odometer,price_ratio,year_diff,odometer_diff,odometer_year_diff
0,1949.0,115148.0,1556.054077,nashville,6.0,bmw,excellent,gas,clean,manual,rwd,mid-size,convertible,orange,tn,229,11503,3044,15219,23545,22364,6155,4833,9411,418,144,3303,bmw_6.0_gas_manual_rwd_mid-size_convertible,15547.0,2003.0,109683.0,1.774426,-54.0,5465.0,67966.000000
1,2013.0,172038.0,17203.800781,state college,8.0,toyota,fair,gas,clean,automatic,rwd,full-size,sedan,silver,pa,12,5727,1784,2404,23545,22364,20497,4833,14635,9259,4300,983,toyota_8.0_gas_automatic_rwd_full-size_sedan,4676.0,2009.0,157886.0,1.010265,4.0,14152.0,63772.390625
2,1998.0,152492.0,6099.680176,wichita,6.0,ford,good,gas,clean,automatic,fwd,full-size,SUV,silver,ks,233,11503,6942,6008,23545,22364,20497,14602,14635,7310,4300,252,ford_6.0_gas_automatic_fwd_full-size_SUV,7797.0,2013.0,128935.0,1.401949,-15.0,23557.0,9714.390625
3,2014.0,104118.0,11568.666992,albany,4.0,ford,excellent,gas,clean,manual,fwd,mid-size,SUV,blue,ny,504,10071,6942,15219,23545,22364,6155,14602,9411,7310,3765,1841,ford_4.0_gas_manual_fwd_mid-size_SUV,8724.0,2010.0,135350.0,1.897409,4.0,-31232.0,-842.046875
4,2005.0,144554.0,8030.777832,redding,6.0,ford,excellent,gas,clean,manual,fwd,mid-size,sedan,red,ca,153,11503,6942,15219,23545,22364,6155,14602,9411,9259,1418,2702,ford_6.0_gas_manual_fwd_mid-size_sedan,6984.5,2013.0,144554.0,0.738492,-8.0,0.0,-5064.781250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27526,2008.0,26660.0,1777.333374,williamsport,6.0,ford,good,gas,clean,automatic,rwd,compact,truck,black,pa,49,11503,6942,6008,23545,22364,20497,4833,3352,3837,10143,983,ford_6.0_gas_automatic_rwd_compact_truck,32212.0,2008.0,26660.0,1.000000,0.0,0.0,-113939.484375
27527,2007.0,108072.0,6754.500000,tulsa,8.0,ford,excellent,gas,clean,automatic,rwd,full-size,pickup,black,ok,166,5727,6942,15219,23545,22364,20497,4833,14635,2118,10143,3303,ford_8.0_gas_automatic_rwd_full-size_pickup,8857.0,2007.0,118969.0,0.609687,0.0,-10897.0,-33087.140625
27528,2019.0,139908.0,34977.000000,rochester,6.0,jeep,like new,gas,clean,automatic,4wd,mid-size,SUV,white,ny,522,11503,1297,3810,23545,22364,20497,8096,9411,7310,3624,1841,jeep_6.0_gas_automatic_4wd_mid-size_SUV,12527.0,2011.0,118883.0,1.774327,8.0,21025.0,71956.851562
27529,2007.0,112326.0,7020.375000,rochester,6.0,jeep,excellent,gas,clean,automatic,4wd,mid-size,sedan,white,ny,522,11503,1297,15219,23545,22364,20497,8096,9411,9259,3624,1841,jeep_6.0_gas_automatic_4wd_mid-size_sedan,4200.5,2013.0,135676.5,0.727056,-6.0,-23350.5,-28833.140625


In [ ]:
# Create a new feature 'car_age'
current_year = datetime.datetime.now().year
train_feat_df['car_age'] = current_year - train_feat_df['ori__year']
test_feat_df['car_age'] = current_year - test_feat_df['ori__year']

In [ ]:
train_feat_df['null_count'] = train_feat_df.isnull().sum(axis=1)
test_feat_df['null_count'] = test_feat_df.isnull().sum(axis=1)

In [ ]:
# Create a robust scaler
scaler = RobustScaler()
train_feat_df['odometer_scaled'] = scaler.fit_transform(train_feat_df[['ori__odometer']])
test_feat_df['odometer_scaled'] = scaler.transform(test_feat_df[['ori__odometer']])

In [ ]:
# Apply log transformation to 'odometer'
train_feat_df['odometer_log'] = np.log1p(train_feat_df['ori__odometer'])
test_feat_df['odometer_log'] = np.log1p(test_feat_df['ori__odometer'])

In [ ]:
# odometer_scaledとcar_ageの交互作用を新たな特徴量として作成します
train_feat_df['odometer_age_interaction'] = train_feat_df['ori__odometer'] * train_feat_df['car_age']
test_feat_df['odometer_age_interaction'] = test_feat_df['ori__odometer'] * test_feat_df['car_age']

In [ ]:
from sklearn.cluster import KMeans
from sklearn.impute import SimpleImputer

# SimpleImputerをインスタンス化（ここでは欠損値を中央値で補完）
imputer = SimpleImputer(strategy='median')

# 補完を行いたい特徴量を選択
features = train_feat_df[['ori__odometer', 'car_age']].copy()

# 欠損値補完
features_imputed = imputer.fit_transform(features)

# KMeansクラスタリングを設定
kmeans = KMeans(n_clusters=10, random_state=0)

# 補完されたデータに基づいてクラスタリング
train_feat_df['odometer_age_cluster'] = kmeans.fit_predict(features_imputed)

# テストデータについても同様に処理
features_test = test_feat_df[['ori__odometer', 'car_age']].copy()
features_test_imputed = imputer.transform(features_test)
test_feat_df['odometer_age_cluster'] = kmeans.predict(features_test_imputed)

# 元のodometer_scaledの欠損値を復元
train_feat_df['ori__odometer'] = features['ori__odometer']
test_feat_df['ori__odometer'] = features_test['ori__odometer']


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [ ]:
train_feat_df

,ori__year,ori__odometer,ori__odometer_per_year,ori__region,ori__cylinders,ori__manufacturer,ori__condition,ori__fuel,ori__title_status,ori__transmission,ori__drive,ori__size,ori__type,ori__paint_color,ori__state,ct__region,ct__cylinders,ct__manufacturer,ct__condition,ct__fuel,ct__title_status,ct__transmission,ct__drive,ct__size,ct__type,ct__paint_color,ct__state,model_id,standard_price,standard_year,standard_odometer,price_ratio,year_diff,odometer_diff,odometer_year_diff,car_age,null_count,odometer_scaled,odometer_log,odometer_age_interaction,odometer_age_cluster
0,1949.0,115148.0,1556.054077,nashville,6.0,bmw,excellent,gas,clean,manual,rwd,mid-size,convertible,orange,tn,229,11503,3044,15219,23545,22364,6155,4833,9411,418,144,3303,bmw_6.0_gas_manual_rwd_mid-size_convertible,15547.0,2003.0,109683.0,1.774426,-54.0,5465.0,67966.000000,74.0,0,0.043103,11.653982,8520952.0,6
1,2013.0,172038.0,17203.800781,state college,8.0,toyota,fair,gas,clean,automatic,rwd,full-size,sedan,silver,pa,12,5727,1784,2404,23545,22364,20497,4833,14635,9259,4300,983,toyota_8.0_gas_automatic_rwd_full-size_sedan,4676.0,2009.0,157886.0,1.010265,4.0,14152.0,63772.390625,10.0,0,0.834621,12.055476,1720380.0,9
2,1998.0,152492.0,6099.680176,wichita,6.0,ford,good,gas,clean,automatic,fwd,full-size,SUV,silver,ks,233,11503,6942,6008,23545,22364,20497,14602,14635,7310,4300,252,ford_6.0_gas_automatic_fwd_full-size_SUV,7797.0,2013.0,128935.0,1.401949,-15.0,23557.0,9714.390625,25.0,0,0.562675,11.934874,3812300.0,0
3,2014.0,104118.0,11568.666992,albany,4.0,ford,excellent,gas,clean,manual,fwd,mid-size,SUV,blue,ny,504,10071,6942,15219,23545,22364,6155,14602,9411,7310,3765,1841,ford_4.0_gas_manual_fwd_mid-size_SUV,8724.0,2010.0,135350.0,1.897409,4.0,-31232.0,-842.046875,9.0,0,-0.110359,11.553289,937062.0,3
4,2005.0,144554.0,8030.777832,redding,6.0,ford,excellent,gas,clean,manual,fwd,mid-size,sedan,red,ca,153,11503,6942,15219,23545,22364,6155,14602,9411,9259,1418,2702,ford_6.0_gas_manual_fwd_mid-size_sedan,6984.5,2013.0,144554.0,0.738492,-8.0,0.0,-5064.781250,18.0,0,0.452233,11.881415,2601972.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27526,2008.0,26660.0,1777.333374,williamsport,6.0,ford,good,gas,clean,automatic,rwd,compact,truck,black,pa,49,11503,6942,6008,23545,22364,20497,4833,3352,3837,10143,983,ford_6.0_gas_automatic_rwd_compact_truck,32212.0,2008.0,26660.0,1.000000,0.0,0.0,-113939.484375,15.0,0,-1.188043,10.190957,399900.0,7
27527,2007.0,108072.0,6754.500000,tulsa,8.0,ford,excellent,gas,clean,automatic,rwd,full-size,pickup,black,ok,166,5727,6942,15219,23545,22364,20497,4833,14635,2118,10143,3303,ford_8.0_gas_automatic_rwd_full-size_pickup,8857.0,2007.0,118969.0,0.609687,0.0,-10897.0,-33087.140625,16.0,0,-0.055346,11.590562,1729152.0,6
27528,2019.0,139908.0,34977.000000,rochester,6.0,jeep,like new,gas,clean,automatic,4wd,mid-size,SUV,white,ny,522,11503,1297,3810,23545,22364,20497,8096,9411,7310,3624,1841,jeep_6.0_gas_automatic_4wd_mid-size_SUV,12527.0,2011.0,118883.0,1.774327,8.0,21025.0,71956.851562,4.0,0,0.387592,11.848747,559632.0,0
27529,2007.0,112326.0,7020.375000,rochester,6.0,jeep,excellent,gas,clean,automatic,4wd,mid-size,sedan,white,ny,522,11503,1297,15219,23545,22364,20497,8096,9411,9259,3624,1841,jeep_6.0_gas_automatic_4wd_mid-size_sedan,4200.5,2013.0,135676.5,0.727056,-6.0,-23350.5,-28833.140625,16.0,0,0.003840,11.629169,1797216.0,6


In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
# 3. Binning features
# Binning 'odometer_scaled' into 10 bins
train_feat_df['odometer_scaled_bin'] = pd.cut(train_feat_df['ori__odometer'], bins=10, labels=False)
test_feat_df['odometer_scaled_bin'] = pd.cut(test_feat_df['ori__odometer'], bins=10, labels=False)

# Binning 'car_age' into 10 bins
train_feat_df['car_age_bin'] = pd.cut(train_feat_df['car_age'], bins=10, labels=False)
test_feat_df['car_age_bin'] = pd.cut(test_feat_df['car_age'], bins=10, labels=False)

# One-hot encoding the binned features
ohe = OneHotEncoder(sparse=False, handle_unknown='ignore')

# 'odometer_scaled_bin'
ohe.fit(train_feat_df[['odometer_scaled_bin']])
train_data_ohe = pd.DataFrame(ohe.transform(train_feat_df[['odometer_scaled_bin']]), columns=[f"odometer_scaled_bin_{c}" for c in ohe.categories_[0]])
test_data_ohe = pd.DataFrame(ohe.transform(test_feat_df[['odometer_scaled_bin']]), columns=[f"odometer_scaled_bin_{c}" for c in ohe.categories_[0]])
train_feat_df = pd.concat([train_feat_df, train_data_ohe], axis=1)
test_feat_df = pd.concat([test_feat_df, test_data_ohe], axis=1)
train_feat_df.drop('odometer_scaled_bin', axis=1, inplace=True)
test_feat_df.drop('odometer_scaled_bin', axis=1, inplace=True)

# 'car_age_bin'
ohe.fit(train_feat_df[['car_age_bin']])
train_data_ohe = pd.DataFrame(ohe.transform(train_feat_df[['car_age_bin']]), columns=[f"car_age_bin_{c}" for c in ohe.categories_[0]])
test_data_ohe = pd.DataFrame(ohe.transform(test_feat_df[['car_age_bin']]), columns=[f"car_age_bin_{c}" for c in ohe.categories_[0]])
train_feat_df = pd.concat([train_feat_df, train_data_ohe], axis=1)
test_feat_df = pd.concat([test_feat_df, test_data_ohe], axis=1)
train_feat_df.drop('car_age_bin', axis=1, inplace=True)
test_feat_df.drop('car_age_bin', axis=1, inplace=True)
train_feat_df.head()

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


,ori__year,ori__odometer,ori__odometer_per_year,ori__region,ori__cylinders,ori__manufacturer,ori__condition,ori__fuel,ori__title_status,ori__transmission,ori__drive,ori__size,ori__type,ori__paint_color,ori__state,ct__region,ct__cylinders,ct__manufacturer,ct__condition,ct__fuel,ct__title_status,ct__transmission,ct__drive,ct__size,ct__type,ct__paint_color,ct__state,model_id,standard_price,standard_year,standard_odometer,price_ratio,year_diff,odometer_diff,odometer_year_diff,car_age,null_count,odometer_scaled,odometer_log,odometer_age_interaction,odometer_age_cluster,odometer_scaled_bin_0.0,odometer_scaled_bin_1.0,odometer_scaled_bin_3.0,odometer_scaled_bin_4.0,odometer_scaled_bin_5.0,odometer_scaled_bin_9.0,odometer_scaled_bin_nan,car_age_bin_0,car_age_bin_1,car_age_bin_2,car_age_bin_3,car_age_bin_4,car_age_bin_5,car_age_bin_6,car_age_bin_7,car_age_bin_8,car_age_bin_9
0,1949.0,115148.0,1556.054077,nashville,6.0,bmw,excellent,gas,clean,manual,rwd,mid-size,convertible,orange,tn,229,11503,3044,15219,23545,22364,6155,4833,9411,418,144,3303,bmw_6.0_gas_manual_rwd_mid-size_convertible,15547.0,2003.0,109683.0,1.774426,-54.0,5465.0,67966.000000,74.0,0,0.043103,11.653982,8520952.0,6,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,2013.0,172038.0,17203.800781,state college,8.0,toyota,fair,gas,clean,automatic,rwd,full-size,sedan,silver,pa,12,5727,1784,2404,23545,22364,20497,4833,14635,9259,4300,983,toyota_8.0_gas_automatic_rwd_full-size_sedan,4676.0,2009.0,157886.0,1.010265,4.0,14152.0,63772.390625,10.0,0,0.834621,12.055476,1720380.0,9,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1998.0,152492.0,6099.680176,wichita,6.0,ford,good,gas,clean,automatic,fwd,full-size,SUV,silver,ks,233,11503,6942,6008,23545,22364,20497,14602,14635,7310,4300,252,ford_6.0_gas_automatic_fwd_full-size_SUV,7797.0,2013.0,128935.0,1.401949,-15.0,23557.0,9714.390625,25.0,0,0.562675,11.934874,3812300.0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2014.0,104118.0,11568.666992,albany,4.0,ford,excellent,gas,clean,manual,fwd,mid-size,SUV,blue,ny,504,10071,6942,15219,23545,22364,6155,14602,9411,7310,3765,1841,ford_4.0_gas_manual_fwd_mid-size_SUV,8724.0,2010.0,135350.0,1.897409,4.0,-31232.0,-842.046875,9.0,0,-0.110359,11.553289,937062.0,3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2005.0,144554.0,8030.777832,redding,6.0,ford,excellent,gas,clean,manual,fwd,mid-size,sedan,red,ca,153,11503,6942,15219,23545,22364,6155,14602,9411,9259,1418,2702,ford_6.0_gas_manual_fwd_mid-size_sedan,6984.5,2013.0,144554.0,0.738492,-8.0,0.0,-5064.781250,18.0,0,0.452233,11.881415,2601972.0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# 1. odometerに対する他の特徴量の比率
# 年ごとの平均odometer値を計算
mean_odometer_per_year = train_feat_df.groupby('ori__year')['ori__odometer'].transform('mean')
# 年ごとのodometerの比率を計算
train_feat_df['odometer_ratio_year'] = train_feat_df['ori__odometer'] / mean_odometer_per_year
# 同様にテストデータセットに対しても行う
mean_odometer_per_year_test = test_feat_df.groupby('ori__year')['ori__odometer'].transform('mean')
test_feat_df['odometer_ratio_year'] = test_feat_df['ori__odometer'] / mean_odometer_per_year_test

# 2. 'odometer'の時間経過による変化
train_feat_df['odometer_per_car_age'] = train_feat_df['ori__odometer'] / train_feat_df['car_age']
test_feat_df['odometer_per_car_age'] = test_feat_df['ori__odometer'] / test_feat_df['car_age']

# 4. 'odometer'のランク
train_feat_df['odometer_rank_region'] = train_feat_df.groupby('ori__region')['ori__odometer'].rank()
test_feat_df['odometer_rank_region'] = test_feat_df.groupby('ori__region')['ori__odometer'].rank()

# 5. 'odometer'のビン化
train_feat_df['odometer_bin_100k'] = pd.cut(train_feat_df['ori__odometer'], bins=[0, 100000, 200000, 300000, np.inf], labels=False)
test_feat_df['odometer_bin_100k'] = pd.cut(test_feat_df['ori__odometer'], bins=[0, 100000, 200000, 300000, np.inf], labels=False)

# 6. 'odometer'の他の数学的変換
train_feat_df['odometer_sqrt'] = np.sqrt(train_feat_df['ori__odometer'])
test_feat_df['odometer_sqrt'] = np.sqrt(test_feat_df['ori__odometer'])

In [ ]:
train_feat_df

,ori__year,ori__odometer,ori__odometer_per_year,ori__region,ori__cylinders,ori__manufacturer,ori__condition,ori__fuel,ori__title_status,ori__transmission,ori__drive,ori__size,ori__type,ori__paint_color,ori__state,ct__region,ct__cylinders,ct__manufacturer,ct__condition,ct__fuel,ct__title_status,ct__transmission,ct__drive,ct__size,ct__type,ct__paint_color,ct__state,model_id,standard_price,standard_year,standard_odometer,price_ratio,year_diff,odometer_diff,odometer_year_diff,car_age,null_count,odometer_scaled,odometer_log,odometer_age_interaction,odometer_age_cluster,odometer_scaled_bin_0.0,odometer_scaled_bin_1.0,odometer_scaled_bin_3.0,odometer_scaled_bin_4.0,odometer_scaled_bin_5.0,odometer_scaled_bin_9.0,odometer_scaled_bin_nan,car_age_bin_0,car_age_bin_1,car_age_bin_2,car_age_bin_3,car_age_bin_4,car_age_bin_5,car_age_bin_6,car_age_bin_7,car_age_bin_8,car_age_bin_9,odometer_ratio_year,odometer_per_car_age,odometer_rank_region,odometer_bin_100k,odometer_sqrt
0,1949.0,115148.0,1556.054077,nashville,6.0,bmw,excellent,gas,clean,manual,rwd,mid-size,convertible,orange,tn,229,11503,3044,15219,23545,22364,6155,4833,9411,418,144,3303,bmw_6.0_gas_manual_rwd_mid-size_convertible,15547.0,2003.0,109683.0,1.774426,-54.0,5465.0,67966.000000,74.0,0,0.043103,11.653982,8520952.0,6,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2.440507,1556.054077,111.0,1.0,339.334656
1,2013.0,172038.0,17203.800781,state college,8.0,toyota,fair,gas,clean,automatic,rwd,full-size,sedan,silver,pa,12,5727,1784,2404,23545,22364,20497,4833,14635,9259,4300,983,toyota_8.0_gas_automatic_rwd_full-size_sedan,4676.0,2009.0,157886.0,1.010265,4.0,14152.0,63772.390625,10.0,0,0.834621,12.055476,1720380.0,9,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.589036,17203.800781,8.0,1.0,414.774628
2,1998.0,152492.0,6099.680176,wichita,6.0,ford,good,gas,clean,automatic,fwd,full-size,SUV,silver,ks,233,11503,6942,6008,23545,22364,20497,14602,14635,7310,4300,252,ford_6.0_gas_automatic_fwd_full-size_SUV,7797.0,2013.0,128935.0,1.401949,-15.0,23557.0,9714.390625,25.0,0,0.562675,11.934874,3812300.0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.068039,6099.680176,170.0,1.0,390.502228
3,2014.0,104118.0,11568.666992,albany,4.0,ford,excellent,gas,clean,manual,fwd,mid-size,SUV,blue,ny,504,10071,6942,15219,23545,22364,6155,14602,9411,7310,3765,1841,ford_4.0_gas_manual_fwd_mid-size_SUV,8724.0,2010.0,135350.0,1.897409,4.0,-31232.0,-842.046875,9.0,0,-0.110359,11.553289,937062.0,3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.991977,11568.666992,273.0,1.0,322.673218
4,2005.0,144554.0,8030.777832,redding,6.0,ford,excellent,gas,clean,manual,fwd,mid-size,sedan,red,ca,153,11503,6942,15219,23545,22364,6155,14602,9411,9259,1418,2702,ford_6.0_gas_manual_fwd_mid-size_sedan,6984.5,2013.0,144554.0,0.738492,-8.0,0.0,-5064.781250,18.0,0,0.452233,11.881415,2601972.0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.966149,8030.777832,114.0,1.0,380.202576
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27526,2008.0,26660.0,1777.333374,williamsport,6.0,ford,good,gas,clean,automatic,rwd,compact,truck,black,pa,49,11503,6942,6008,23545,22364,20497,4833,3352,3837,10143,983,ford_6.0_gas_automatic_rwd_compact_truck,32212.0,2008.0,26660.0,1.000000,0.0,0.0,-113939.484375,15.0,0,-1.188043,10.190957,399900.0,7,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.189617,1777.333374,4.0,0.0,163.278900
27527,2007.0,108072.0,6754.500000,tulsa,8.0,ford,excellent,gas,clean,automatic,rwd,full-size,pickup,black,ok,166,5727,6942,15219,23545,22364,20497,4833,14635,2118,10143,3303,ford_8.0_gas_automatic_rwd_full-size_pickup,8857.0,2007.0,118969.0,0.609687,0.0,-10897.0,-33087.140625,16.0,0,-0.055346,11

In [ ]:
test_feat_df

,ori__year,ori__odometer,ori__odometer_per_year,ori__region,ori__cylinders,ori__manufacturer,ori__condition,ori__fuel,ori__title_status,ori__transmission,ori__drive,ori__size,ori__type,ori__paint_color,ori__state,ct__region,ct__cylinders,ct__manufacturer,ct__condition,ct__fuel,ct__title_status,ct__transmission,ct__drive,ct__size,ct__type,ct__paint_color,ct__state,model_id,standard_year,standard_odometer,year_diff,odometer_diff,odometer_year_diff,car_age,null_count,odometer_scaled,odometer_log,odometer_age_interaction,odometer_age_cluster,odometer_scaled_bin_0.0,odometer_scaled_bin_1.0,odometer_scaled_bin_3.0,odometer_scaled_bin_4.0,odometer_scaled_bin_5.0,odometer_scaled_bin_9.0,odometer_scaled_bin_nan,car_age_bin_0,car_age_bin_1,car_age_bin_2,car_age_bin_3,car_age_bin_4,car_age_bin_5,car_age_bin_6,car_age_bin_7,car_age_bin_8,car_age_bin_9,odometer_ratio_year,odometer_per_car_age,odometer_rank_region,odometer_bin_100k,odometer_sqrt
0,2015.0,92553.0,11569.125000,western slope,4.0,chevrolet,excellent,gas,clean,automatic,fwd,full-size,SUV,red,co,138,10071,3807,15219,23545,22364,20497,14602,14635,7310,1418,3303,chevrolet_4.0_gas_automatic_fwd_full-size_SUV,2012.0,105178.0,3.0,-12625.0,-6103.218750,8.0,0,-0.271264,11.435548,740424.0,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.963558,11569.125000,30.0,0.0,304.225250
1,2013.0,134385.0,13438.500000,roseburg,4.0,nissan,like new,gas,salvage,automatic,fwd,mid-size,sedan,black,or,21,10071,1003,3810,23545,985,20497,14602,9411,9259,10143,577,nissan_4.0_gas_automatic_fwd_mid-size_sedan,2014.0,107921.0,-1.0,26464.0,26119.390625,10.0,0,0.310750,11.808472,1343850.0,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.245104,13438.500000,11.0,1.0,366.585602
2,2011.0,102489.0,8540.750000,akron / canton,4.0,volkswagen,good,gas,clean,automatic,fwd,full-size,sedan,black,oh,102,10071,773,6008,23545,22364,20497,14602,14635,9259,10143,849,volkswagen_4.0_gas_automatic_fwd_full-size_sedan,2013.0,115055.0,-2.0,-12566.0,-12658.710938,12.0,0,-0.133024,11.537520,1229868.0,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.901052,8540.750000,50.0,1.0,320.139038
3,2016.0,64310.0,9187.142578,denver,6.0,jeep,excellent,diesel,clean,automatic,4wd,mid-size,SUV,red,co,246,11503,1297,15219,2639,22364,20497,8096,9411,7310,1418,705,jeep_6.0_diesel_automatic_4wd_mid-size_SUV,2013.0,114543.0,3.0,-50233.0,-20847.601562,7.0,0,-0.664213,11.071486,450170.0,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.726973,9187.142578,39.0,0.0,253.594162
4,1999.0,180839.0,7534.958496,hickory / lenoir,8.0,honda,excellent,gas,rebuilt,automatic,4wd,mid-size,SUV,silver,nc,66,5727,1742,15219,23545,3004,20497,8096,9411,7310,4300,636,honda_8.0_gas_automatic_4wd_mid-size_SUV,NaN,NaN,NaN,NaN,30437.593750,24.0,4,0.957071,12.105368,4340136.0,9,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.207323,7534.958496,62.0,1.0,425.251678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27532,2016.0,90902.0,12986.000000,great falls,6.0,gmc,excellent,gas,rebuilt,automatic,fwd,full-size,truck,black,mt,11,11503,370,15219,23545,3004,20497,14602,14635,3837,10143,249,gmc_6.0_gas_automatic_fwd_full-size_truck,2011.0,173029.0,5.0,-82127.0,5744.398438,7.0,0,-0.294235,11.417548,636314.0,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.027575,12986.000000,8.0,0.0,301.499573
27533,2012.0,27234.0,2475.818115,las vegas,4.0,nissan,excellent,gas,rebuilt,automatic,fwd,mid-size,sedan,white,nv,429,10071,1003,15219,23545,3004,20497,14602,9411,9259,3624,369,nissan_4.0_gas_automatic_fwd_mid-size_sedan,2014.0,107921.0,-2.0,-80687.0,-84914.156250,11.0,0,-1.180057,10.212258,299574.0,7,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

In [ ]:


# List of categorical columns to apply label encoding
label_cols = ['ori__region']
onehot_cols = ['ori__manufacturer', 'ori__condition', 'ori__fuel', 'ori__title_status',
               'ori__transmission', 'ori__drive', 'ori__size', 'ori__type', 'ori__paint_color', 'ori__state']

# Create a copy of the dataframes
train_data_encoded = train_feat_df.copy()
test_data_encoded = test_feat_df.copy()

# Instantiate the LabelEncoder and OneHotEncoder
le = LabelEncoder()
ohe = OneHotEncoder(sparse=False)

# Apply label encoding
for col in label_cols:
    # Fit the encoder on the training data
    le.fit(train_data_encoded[col])

    # Transform the data
    train_data_encoded['label_' + col] = le.transform(train_data_encoded[col])
    test_data_encoded['label_' + col] = le.transform(test_data_encoded[col])

# Apply one-hot encoding
for col in onehot_cols:
    # Fit the encoder on the training data
    ohe.fit(train_data_encoded[[col]])

    # Transform the data
    train_data_ohe = pd.DataFrame(ohe.transform(train_data_encoded[[col]]), columns=[f"{col}_{c}" for c in ohe.categories_[0]])
    test_data_ohe = pd.DataFrame(ohe.transform(test_data_encoded[[col]]), columns=[f"{col}_{c}" for c in ohe.categories_[0]])

    # Drop the original column
    #train_data_encoded.drop(col, axis=1, inplace=True)
    #test_data_encoded.drop(col, axis=1, inplace=True)

    # Concatenate the one-hot encoded columns
    train_data_encoded = pd.concat([train_data_encoded, train_data_ohe], axis=1)
    test_data_encoded = pd.concat([test_data_encoded, test_data_ohe], axis=1)



/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_outp

In [ ]:
train_feat_df = train_data_encoded
test_feat_df = test_data_encoded

In [ ]:
train_feat_df


,ori__year,ori__odometer,ori__odometer_per_year,ori__region,ori__cylinders,ori__manufacturer,ori__condition,ori__fuel,ori__title_status,ori__transmission,ori__drive,ori__size,ori__type,ori__paint_color,ori__state,ct__region,ct__cylinders,ct__manufacturer,ct__condition,ct__fuel,ct__title_status,ct__transmission,ct__drive,ct__size,ct__type,ct__paint_color,ct__state,model_id,standard_price,standard_year,standard_odometer,price_ratio,year_diff,odometer_diff,odometer_year_diff,car_age,null_count,odometer_scaled,odometer_log,odometer_age_interaction,odometer_age_cluster,odometer_scaled_bin_0.0,odometer_scaled_bin_1.0,odometer_scaled_bin_3.0,odometer_scaled_bin_4.0,odometer_scaled_bin_5.0,odometer_scaled_bin_9.0,odometer_scaled_bin_nan,car_age_bin_0,car_age_bin_1,car_age_bin_2,car_age_bin_3,car_age_bin_4,car_age_bin_5,car_age_bin_6,car_age_bin_7,car_age_bin_8,car_age_bin_9,odometer_ratio_year,odometer_per_car_age,odometer_rank_region,odometer_bin_100k,odometer_sqrt,label_ori__region,ori__manufacturer_acura,ori__manufacturer_alfa-romeo,ori__manufacturer_aston-martin,ori__manufacturer_audi,ori__manufacturer_bmw,ori__manufacturer_buick,ori__manufacturer_cadillac,ori__manufacturer_chevrolet,ori__manufacturer_chrysler,ori__manufacturer_cura,ori__manufacturer_dodge,ori__manufacturer_fiat,ori__manufacturer_ford,ori__manufacturer_gmc,ori__manufacturer_honda,ori__manufacturer_hrysler,ori__manufacturer_hyundai,ori__manufacturer_infiniti,ori__manufacturer_jaguar,ori__manufacturer_jeep,ori__manufacturer_kia,ori__manufacturer_land rover,ori__manufacturer_lexus,ori__manufacturer_lincoln,ori__manufacturer_mazda,ori__manufacturer_mercedes-benz,ori__manufacturer_mercury,ori__manufacturer_mini,ori__manufacturer_mitsubishi,ori__manufacturer_nissan,ori__manufacturer_pontiac,ori__manufacturer_porsche,ori__manufacturer_ram,ori__manufacturer_rover,ori__manufacturer_saturn,ori__manufacturer_subaru,ori__manufacturer_toyota,ori__manufacturer_volkswagen,ori__manufacturer_volvo,ori__condition_excellent,ori__condition_fair,ori__condition_good,ori__condition_like new,ori__condition_new,ori__condition_salvage,ori__fuel_diesel,ori__fuel_electric,ori__fuel_gas,ori__fuel_hybrid,ori__fuel_other,ori__fuel_nan,ori__title_status_clean,ori__title_status_lien,ori__title_status_missing,ori__title_status_parts only,ori__title_status_rebuilt,ori__title_status_salvage,ori__title_status_nan,ori__transmission_automatic,ori__transmission_manual,ori__transmission_other,ori__drive_4wd,ori__drive_fwd,ori__drive_rwd,ori__size_compact,ori__size_full-size,ori__size_mid-size,ori__size_sub-compact,ori__type_SUV,ori__type_bus,ori__type_convertible,ori__type_coupe,ori__type_hatchback,ori__type_mini-van,ori__type_offroad,ori__type_other,ori__type_pickup,ori__type_sedan,ori__type_truck,ori__type_van,ori__type_wagon,ori__type_nan,ori__paint_color_black,ori__paint_color_blue,ori__paint_color_brown,ori__paint_color_custom,ori__paint_color_green,ori__paint_color_grey,ori__paint_color_orange,ori__paint_color_purple,ori__paint_color_red,ori__paint_color_silver,ori__paint_color_white,ori__paint_color_yellow,ori__state_ak,ori__state_al,ori__state_ar,ori__state_az,ori__state_ca,ori__state_co,ori__state_ct,ori__state_dc,ori__state_de,ori__state_fl,ori__state_ga,ori__state_hi,ori__state_ia,ori__state_id,ori__state_il,ori__state_in,ori__state_ks,ori__state_ky,ori__state_la,ori__state_ma,ori__state_md,ori__state_me,ori__state_mi,ori__state_mn,ori__state_mo,ori__state_ms,ori__state_mt,ori__state_nc,ori__state_nd,ori__state_ne,ori__state_nh,ori__state_nj,ori__state_nm,ori__state_nv,ori__state_ny,ori__state_oh,ori__state_ok,ori__state_or,ori__state_pa,ori__state_ri,ori__state_sc,ori__state_sd,ori__state_tn,ori__state_tx,ori__state_ut,ori__state_va,ori__state_vt,ori__state_wa,ori__state_wi,ori__state_wv,ori__state_wy,ori__state_nan
0,1949.0,115148.0,1556.054077,nashville,6.0,bmw,excellent,gas,clean,manual,rwd,mid-size,convertible,orange,tn,229,11503,3044,15219,23545,22364,6155,4833,9411,418,144,3303,bmw_6.0_

In [ ]:
test_feat_df

,ori__year,ori__odometer,ori__odometer_per_year,ori__region,ori__cylinders,ori__manufacturer,ori__condition,ori__fuel,ori__title_status,ori__transmission,ori__drive,ori__size,ori__type,ori__paint_color,ori__state,ct__region,ct__cylinders,ct__manufacturer,ct__condition,ct__fuel,ct__title_status,ct__transmission,ct__drive,ct__size,ct__type,ct__paint_color,ct__state,model_id,standard_year,standard_odometer,year_diff,odometer_diff,odometer_year_diff,car_age,null_count,odometer_scaled,odometer_log,odometer_age_interaction,odometer_age_cluster,odometer_scaled_bin_0.0,odometer_scaled_bin_1.0,odometer_scaled_bin_3.0,odometer_scaled_bin_4.0,odometer_scaled_bin_5.0,odometer_scaled_bin_9.0,odometer_scaled_bin_nan,car_age_bin_0,car_age_bin_1,car_age_bin_2,car_age_bin_3,car_age_bin_4,car_age_bin_5,car_age_bin_6,car_age_bin_7,car_age_bin_8,car_age_bin_9,odometer_ratio_year,odometer_per_car_age,odometer_rank_region,odometer_bin_100k,odometer_sqrt,label_ori__region,ori__manufacturer_acura,ori__manufacturer_alfa-romeo,ori__manufacturer_aston-martin,ori__manufacturer_audi,ori__manufacturer_bmw,ori__manufacturer_buick,ori__manufacturer_cadillac,ori__manufacturer_chevrolet,ori__manufacturer_chrysler,ori__manufacturer_cura,ori__manufacturer_dodge,ori__manufacturer_fiat,ori__manufacturer_ford,ori__manufacturer_gmc,ori__manufacturer_honda,ori__manufacturer_hrysler,ori__manufacturer_hyundai,ori__manufacturer_infiniti,ori__manufacturer_jaguar,ori__manufacturer_jeep,ori__manufacturer_kia,ori__manufacturer_land rover,ori__manufacturer_lexus,ori__manufacturer_lincoln,ori__manufacturer_mazda,ori__manufacturer_mercedes-benz,ori__manufacturer_mercury,ori__manufacturer_mini,ori__manufacturer_mitsubishi,ori__manufacturer_nissan,ori__manufacturer_pontiac,ori__manufacturer_porsche,ori__manufacturer_ram,ori__manufacturer_rover,ori__manufacturer_saturn,ori__manufacturer_subaru,ori__manufacturer_toyota,ori__manufacturer_volkswagen,ori__manufacturer_volvo,ori__condition_excellent,ori__condition_fair,ori__condition_good,ori__condition_like new,ori__condition_new,ori__condition_salvage,ori__fuel_diesel,ori__fuel_electric,ori__fuel_gas,ori__fuel_hybrid,ori__fuel_other,ori__fuel_nan,ori__title_status_clean,ori__title_status_lien,ori__title_status_missing,ori__title_status_parts only,ori__title_status_rebuilt,ori__title_status_salvage,ori__title_status_nan,ori__transmission_automatic,ori__transmission_manual,ori__transmission_other,ori__drive_4wd,ori__drive_fwd,ori__drive_rwd,ori__size_compact,ori__size_full-size,ori__size_mid-size,ori__size_sub-compact,ori__type_SUV,ori__type_bus,ori__type_convertible,ori__type_coupe,ori__type_hatchback,ori__type_mini-van,ori__type_offroad,ori__type_other,ori__type_pickup,ori__type_sedan,ori__type_truck,ori__type_van,ori__type_wagon,ori__type_nan,ori__paint_color_black,ori__paint_color_blue,ori__paint_color_brown,ori__paint_color_custom,ori__paint_color_green,ori__paint_color_grey,ori__paint_color_orange,ori__paint_color_purple,ori__paint_color_red,ori__paint_color_silver,ori__paint_color_white,ori__paint_color_yellow,ori__state_ak,ori__state_al,ori__state_ar,ori__state_az,ori__state_ca,ori__state_co,ori__state_ct,ori__state_dc,ori__state_de,ori__state_fl,ori__state_ga,ori__state_hi,ori__state_ia,ori__state_id,ori__state_il,ori__state_in,ori__state_ks,ori__state_ky,ori__state_la,ori__state_ma,ori__state_md,ori__state_me,ori__state_mi,ori__state_mn,ori__state_mo,ori__state_ms,ori__state_mt,ori__state_nc,ori__state_nd,ori__state_ne,ori__state_nh,ori__state_nj,ori__state_nm,ori__state_nv,ori__state_ny,ori__state_oh,ori__state_ok,ori__state_or,ori__state_pa,ori__state_ri,ori__state_sc,ori__state_sd,ori__state_tn,ori__state_tx,ori__state_ut,ori__state_va,ori__state_vt,ori__state_wa,ori__state_wi,ori__state_wv,ori__state_wy,ori__state_nan
0,2015.0,92553.0,11569.125000,western slope,4.0,chevrolet,excellent,gas,clean,automatic,fwd,full-size,SUV,red,co,138,10071,3807,15219,23545,22364,20497,14602,14635,7310,1418,3303,chevrolet_4.0_gas_automatic

In [ ]:
from sklearn.model_selection import StratifiedKFold

def get_stratified_cv(df, n_fold=5):
    skf = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=71)
    return list(skf.split(df, df['ori__manufacturer']))

n_fold = 5
cv = get_stratified_cv(train_feat_df, n_fold=n_fold)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:725: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


In [ ]:
train_feat_df

,ori__year,ori__odometer,ori__odometer_per_year,ori__region,ori__cylinders,ori__manufacturer,ori__condition,ori__fuel,ori__title_status,ori__transmission,ori__drive,ori__size,ori__type,ori__paint_color,ori__state,ct__region,ct__cylinders,ct__manufacturer,ct__condition,ct__fuel,ct__title_status,ct__transmission,ct__drive,ct__size,ct__type,ct__paint_color,ct__state,model_id,standard_price,standard_year,standard_odometer,price_ratio,year_diff,odometer_diff,odometer_year_diff,car_age,null_count,odometer_scaled,odometer_log,odometer_age_interaction,odometer_age_cluster,odometer_scaled_bin_0.0,odometer_scaled_bin_1.0,odometer_scaled_bin_3.0,odometer_scaled_bin_4.0,odometer_scaled_bin_5.0,odometer_scaled_bin_9.0,odometer_scaled_bin_nan,car_age_bin_0,car_age_bin_1,car_age_bin_2,car_age_bin_3,car_age_bin_4,car_age_bin_5,car_age_bin_6,car_age_bin_7,car_age_bin_8,car_age_bin_9,odometer_ratio_year,odometer_per_car_age,odometer_rank_region,odometer_bin_100k,odometer_sqrt,label_ori__region,ori__manufacturer_acura,ori__manufacturer_alfa-romeo,ori__manufacturer_aston-martin,ori__manufacturer_audi,ori__manufacturer_bmw,ori__manufacturer_buick,ori__manufacturer_cadillac,ori__manufacturer_chevrolet,ori__manufacturer_chrysler,ori__manufacturer_cura,ori__manufacturer_dodge,ori__manufacturer_fiat,ori__manufacturer_ford,ori__manufacturer_gmc,ori__manufacturer_honda,ori__manufacturer_hrysler,ori__manufacturer_hyundai,ori__manufacturer_infiniti,ori__manufacturer_jaguar,ori__manufacturer_jeep,ori__manufacturer_kia,ori__manufacturer_land rover,ori__manufacturer_lexus,ori__manufacturer_lincoln,ori__manufacturer_mazda,ori__manufacturer_mercedes-benz,ori__manufacturer_mercury,ori__manufacturer_mini,ori__manufacturer_mitsubishi,ori__manufacturer_nissan,ori__manufacturer_pontiac,ori__manufacturer_porsche,ori__manufacturer_ram,ori__manufacturer_rover,ori__manufacturer_saturn,ori__manufacturer_subaru,ori__manufacturer_toyota,ori__manufacturer_volkswagen,ori__manufacturer_volvo,ori__condition_excellent,ori__condition_fair,ori__condition_good,ori__condition_like new,ori__condition_new,ori__condition_salvage,ori__fuel_diesel,ori__fuel_electric,ori__fuel_gas,ori__fuel_hybrid,ori__fuel_other,ori__fuel_nan,ori__title_status_clean,ori__title_status_lien,ori__title_status_missing,ori__title_status_parts only,ori__title_status_rebuilt,ori__title_status_salvage,ori__title_status_nan,ori__transmission_automatic,ori__transmission_manual,ori__transmission_other,ori__drive_4wd,ori__drive_fwd,ori__drive_rwd,ori__size_compact,ori__size_full-size,ori__size_mid-size,ori__size_sub-compact,ori__type_SUV,ori__type_bus,ori__type_convertible,ori__type_coupe,ori__type_hatchback,ori__type_mini-van,ori__type_offroad,ori__type_other,ori__type_pickup,ori__type_sedan,ori__type_truck,ori__type_van,ori__type_wagon,ori__type_nan,ori__paint_color_black,ori__paint_color_blue,ori__paint_color_brown,ori__paint_color_custom,ori__paint_color_green,ori__paint_color_grey,ori__paint_color_orange,ori__paint_color_purple,ori__paint_color_red,ori__paint_color_silver,ori__paint_color_white,ori__paint_color_yellow,ori__state_ak,ori__state_al,ori__state_ar,ori__state_az,ori__state_ca,ori__state_co,ori__state_ct,ori__state_dc,ori__state_de,ori__state_fl,ori__state_ga,ori__state_hi,ori__state_ia,ori__state_id,ori__state_il,ori__state_in,ori__state_ks,ori__state_ky,ori__state_la,ori__state_ma,ori__state_md,ori__state_me,ori__state_mi,ori__state_mn,ori__state_mo,ori__state_ms,ori__state_mt,ori__state_nc,ori__state_nd,ori__state_ne,ori__state_nh,ori__state_nj,ori__state_nm,ori__state_nv,ori__state_ny,ori__state_oh,ori__state_ok,ori__state_or,ori__state_pa,ori__state_ri,ori__state_sc,ori__state_sd,ori__state_tn,ori__state_tx,ori__state_ut,ori__state_va,ori__state_vt,ori__state_wa,ori__state_wi,ori__state_wv,ori__state_wy,ori__state_nan
0,1949.0,115148.0,1556.054077,nashville,6.0,bmw,excellent,gas,clean,manual,rwd,mid-size,convertible,orange,tn,229,11503,3044,15219,23545,22364,6155,4833,9411,418,144,3303,bmw_6.0_

In [ ]:
train_df

,id,region,year,manufacturer,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color,state,price,odometer_per_year,model_id
0,0,nashville,1949,bmw,excellent,6.0,gas,115148.0,clean,manual,rwd,mid-size,convertible,orange,NaN,27587,1556.054054,bmw_6.0_gas_manual_rwd_mid-size_convertible
1,1,state college,2013,toyota,fair,8.0,gas,172038.0,clean,automatic,rwd,full-size,sedan,silver,pa,4724,17203.800000,toyota_8.0_gas_automatic_rwd_full-size_sedan
2,2,wichita,1998,ford,good,6.0,gas,152492.0,clean,automatic,fwd,full-size,SUV,silver,ks,10931,6099.680000,ford_6.0_gas_automatic_fwd_full-size_SUV
3,3,albany,2014,ford,excellent,4.0,gas,104118.0,clean,manual,fwd,mid-size,SUV,blue,ny,16553,11568.666667,ford_4.0_gas_manual_fwd_mid-size_SUV
4,4,redding,2005,ford,excellent,6.0,gas,144554.0,clean,manual,fwd,mid-size,sedan,red,ca,5158,8030.777778,ford_6.0_gas_manual_fwd_mid-size_sedan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27526,27527,williamsport,2008,ford,good,6.0,gas,26660.0,clean,automatic,rwd,compact,truck,black,pa,32212,1777.333333,ford_6.0_gas_automatic_rwd_compact_truck
27527,27528,tulsa,2007,ford,excellent,8.0,gas,108072.0,clean,automatic,rwd,full-size,pickup,black,NaN,5400,6754.500000,ford_8.0_gas_automatic_rwd_full-size_pickup
27528,27529,rochester,2019,jeep,like new,6.0,gas,139908.0,clean,automatic,4wd,mid-size,SUV,white,ny,22227,34977.000000,jeep_6.0_gas_automatic_4wd_mid-size_SUV
27529,27530,rochester,2007,jeep,excellent,6.0,gas,112326.0,clean,automatic,4wd,mid-size,sedan,white,ny,3054,7020.375000,jeep_6.0_gas_automatic_4wd_mid-size_sedan


In [ ]:
#train_feat_df = train_feat_df.fillna(train_feat_df.median())

In [ ]:
# infを最大値で置き換える
#train_feat_df = train_feat_df.replace([np.inf, -np.inf], np.nan)
#train_feat_df = train_feat_df.fillna(train_feat_df.max())

In [ ]:
def fit_lgbm(X, y, test_data, cv, categorical_cols: list = None, params: dict = None, verbose: int = 50):
    if params is None:
        params = {}

    models = []
    target_encoders = []
    transformed_test_data_list = []
    n_records = len(X)
    oof_pred = np.zeros((n_records,), dtype=np.float32)

    for i, (idx_train, idx_valid) in enumerate(cv):
        X_tr, y_tr = X.iloc[idx_train], y.iloc[idx_train]
        X_val, y_val = X.iloc[idx_valid], y.iloc[idx_valid]

        transformed_test_data = test_data.copy()

        # 価格の平均値によるターゲットエンコーディングを適用するカラム
        #te_cols_price_mean = ['ori__region',"ori__cylinders", "ori__manufacturer","ori__condition","ori__fuel","ori__title_status","ori__transmission","ori__drive","ori__size","ori__type","ori__paint_color","ori__state"]
        te_cols_price_mean = ['ori__manufacturer', 'ori__size', 'ori__type', 'ori__fuel']

        for col in te_cols_price_mean:
            te = TargetEncoder(target_type="continuous", random_state=88)
            te.fit(X_tr[[col]], y_tr)
            X_tr['te_mean_price_' + col] = te.transform(X_tr[[col]])
            X_val['te_mean_price_' + col] = te.transform(X_val[[col]])
            transformed_test_data['te_mean_price_' + col] = te.transform(transformed_test_data[[col]])


        # 価格比の平均値によるターゲットエンコーディングを適用するカラム
        te_cols_price_ratio_mean = ['ori__condition', 'ori__title_status', 'ori__state', 'ori__region', 'ori__paint_color']

        for col in te_cols_price_ratio_mean:
            te = TargetEncoder(target_type="continuous", random_state=88)
            te.fit(X_tr[[col]], X_tr['price_ratio'])  # price_ratioが計算されている前提
            X_tr['te_mean_price_ratio_' + col] = te.transform(X_tr[[col]])
            X_val['te_mean_price_ratio_' + col] = te.transform(X_val[[col]])
            transformed_test_data['te_mean_price_ratio_' + col] = te.transform(transformed_test_data[[col]])

        # 価格の標準偏差によるターゲットエンコーディングを適用するカラム
        te_cols_price_std = ['ori__manufacturer', 'ori__condition', 'ori__fuel', 'ori__title_status', 'ori__transmission', 'ori__size', 'ori__type', 'ori__state', 'model_id']


        X_tr = pd.concat([X_tr,y_tr],axis=1)
        # 各カラムの標準偏差を計算
        for col in te_cols_price_std:
            std_values = X_tr.groupby(col)['price'].std().rename('std_price_' + col)
            X_tr = X_tr.join(std_values, on=col)
            X_val = X_val.join(std_values, on=col)  # 検証データとテストデータにも同様に適用
            transformed_test_data = transformed_test_data.join(std_values, on=col)



        drop_cols = ['ori__region', 'ori__manufacturer', 'ori__condition', 'ori__fuel', 'ori__title_status',
               'ori__transmission', 'ori__drive', 'ori__size', 'ori__type', 'ori__paint_color', 'ori__state', 'model_id']
        X_tr.drop(drop_cols+['price_ratio', 'standard_price', 'price'], axis=1, inplace=True)
        X_val.drop(drop_cols+['price_ratio', 'standard_price'], axis=1, inplace=True)
        transformed_test_data.drop(drop_cols, axis=1, inplace=True)
        #X_tr.to_csv(f'/content/drive/MyDrive/Colab Notebooks/signate2023/exp00025/signate-models/X_tr_{i}.csv', index=False)
        #transformed_test_data.to_csv(f'/content/drive/MyDrive/Colab Notebooks/signate2023/exp00025/signate-models/transformed_test_data_{i}.csv', index=False)

        transformed_test_data_list.append(transformed_test_data)

        # Save the fitted target encoder
        target_encoders.append(te)

        model = lgb.LGBMRegressor(**params)
        model.fit(
            X_tr,
            y_tr,
            categorical_feature=categorical_cols,
            eval_set=[(X_val, y_val)],
            callbacks=[lgb.early_stopping(100, verbose=verbose)],
        )
        pred_i = model.predict(X_val)
        oof_pred[idx_valid] = pred_i
        models.append(model)
        score = mean_absolute_percentage_error(y_val, pred_i)
        print(f" - fold{i + 1} - {score:.4f}")

    score = mean_absolute_percentage_error(y, oof_pred)
    print("=" * 50)
    print(f"FINISHI: Whole Score: {score:.4f}")
    return score, oof_pred, models, target_encoders, transformed_test_data_list



# optunaを使ったパラメータチューニング
def tuning(train_feat_df, train_df, cv):
     def objective(trial):
         max_depth = trial.suggest_int("max_depth", 1, 10)
         num_leaves = trial.suggest_int("num_leaves", 2, 2**max_depth)
         colsample_bytree = trial.suggest_uniform("colsample_bytree", 0.1, 1.0)
         subsample = trial.suggest_uniform("subsample", 0.1, 1.0)
         learning_rate = trial.suggest_float('learning_rate',0.03,0.1)
         n_estimators = trial.suggest_int('n_estimators', 100, 50000)

         params = {
             "objective": "mape",
             "n_estimators": n_estimators,
             "learning_rate": learning_rate,
             "max_depth": max_depth,
             "num_leaves": num_leaves,
             "colsample_bytree": colsample_bytree,
             "subsample": subsample,
             "metric": "mape",
             "importance_type": "gain",
             "random_state": 88,
         }

         te_cols = [
            'ori__region',
            "ori__cylinders",
            "ori__manufacturer",
            "ori__condition",
            "ori__fuel",
            "ori__title_status",
            "ori__transmission",
            "ori__drive",
            "ori__size",
            "ori__type",
            "ori__paint_color",
            "ori__state",
            ]

         score, _, _, _, _ = fit_lgbm(
             X=train_feat_df, y=train_df["price"], test_data=test_feat_df, cv=cv,  categorical_cols=[], params=params, verbose=-1
         )
         return score

     study = optuna.create_study(direction="minimize")
     study.optimize(objective, n_trials=50)
     print("Number of finished trials:", len(study.trials))
     print("Best trial:", study.best_trial.params)
     return study.best_trial.params

In [ ]:
Best trial: {'max_depth': 7, 'num_leaves': 128, 'colsample_bytree': 0.46229496807951187, 'subsample': 0.5987637326011412, 'learning_rate': 0.035785358237282267, 'n_estimators': 30704}
FINISHI: Whole Score: 0.4439

SyntaxError: ignored

In [ ]:
train_df["price"]

0        27587
1         4724
2        10931
3        16553
4         5158
         ...  
27526    32212
27527     5400
27528    22227
27529     3054
27530    20801
Name: price, Length: 27531, dtype: int64

In [ ]:
lgbm_params = {
    "objective": "mape",
    "metrics": "mape",
    "n_estimators": 30704,
    "learning_rate": 0.035785358237282267,
    "max_depth": 7,
    "num_leaves": 128,
    "colsample_bytree": 0.46229496807951187,
    "subsample":   0.5987637326011412,
    "importance_type": "gain",
    "random_state": 88,
}

te_cols = [
            'ori__region',
            "ori__cylinders",
            "ori__manufacturer",
            "ori__condition",
            "ori__fuel",
            "ori__title_status",
            "ori__transmission",
            "ori__drive",
            "ori__size",
            "ori__type",
            "ori__paint_color",
            "ori__state",
            ]

# training
score, oof_lgb, models_lgb, target_encoders, transformed_test_data_list  = fit_lgbm(
    train_feat_df,
    y=train_df["price"],
    test_data=test_feat_df,
    categorical_cols=[],
    params=lgbm_params,
    cv=cv,
    verbose=False,
)

<ipython-input-54-3886878697dd>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_tr['te_mean_price_' + col] = te.transform(X_tr[[col]])
<ipython-input-54-3886878697dd>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_val['te_mean_price_' + col] = te.transform(X_val[[col]])
<ipython-input-54-3886878697dd>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://p

 - fold1 - 0.4464


<ipython-input-54-3886878697dd>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_tr['te_mean_price_' + col] = te.transform(X_tr[[col]])
<ipython-input-54-3886878697dd>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_val['te_mean_price_' + col] = te.transform(X_val[[col]])
<ipython-input-54-3886878697dd>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://p

 - fold2 - 0.4442


<ipython-input-54-3886878697dd>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_tr['te_mean_price_' + col] = te.transform(X_tr[[col]])
<ipython-input-54-3886878697dd>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_val['te_mean_price_' + col] = te.transform(X_val[[col]])
<ipython-input-54-3886878697dd>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://p

 - fold3 - 0.4424


<ipython-input-54-3886878697dd>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_tr['te_mean_price_' + col] = te.transform(X_tr[[col]])
<ipython-input-54-3886878697dd>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_val['te_mean_price_' + col] = te.transform(X_val[[col]])
<ipython-input-54-3886878697dd>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://p

 - fold4 - 0.4470


<ipython-input-54-3886878697dd>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_tr['te_mean_price_' + col] = te.transform(X_tr[[col]])
<ipython-input-54-3886878697dd>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_val['te_mean_price_' + col] = te.transform(X_val[[col]])
<ipython-input-54-3886878697dd>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://p

 - fold5 - 0.4453
FINISHI: Whole Score: 0.4451


In [ ]:
base FINISHI: Whole Score: 0.4438
1 FINISHI: Whole Score: 0.4438

In [ ]:
train_feat_df

In [ ]:
feat_cat_cols = train_feat_df.select_dtypes(include="category").columns.tolist()


lgbm_params = tuning(train_feat_df, train_df, cv)

lgbm_params.update(
    {
    "objective": "mape",
    "metrics": "mape",
    "importance_type": "gain",
    "random_state": 88,
    }
)

te_cols = [
            'ori__region',
            "ori__cylinders",
            "ori__manufacturer",
            "ori__condition",
            "ori__fuel",
            "ori__title_status",
            "ori__transmission",
            "ori__drive",
            "ori__size",
            "ori__type",
            "ori__paint_color",
            "ori__state",
            ]

# training
score, oof_lgb, models_lgb, target_encoderb, transformed_test_data_list = fit_lgbm(
    train_feat_df,
    y=train_df["price"],
    test_data=test_feat_df,
    categorical_cols=[],
    params=lgbm_params,
    cv=cv,
    verbose=False,
)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# Create a copy of the dataframe
train_data_transformed = train_feat_df.copy()

# Apply target encoding to the copy of the dataframe
for col in te_cols:
    # Create a new instance of TargetEncoder and fit it to the whole data
    te = TargetEncoder(target_type="continuous", random_state=88)
    te.fit(train_data_transformed[[col]], train_df["price"])
    train_data_transformed['te_' + col] = te.transform(train_data_transformed[[col]])

# Drop the original columns
drop_cols = ['ori__region', 'ori__manufacturer', 'ori__condition', 'ori__fuel', 'ori__title_status',
       'ori__transmission', 'ori__drive', 'ori__size', 'ori__type', 'ori__paint_color', 'ori__state']
train_data_transformed.drop(drop_cols, axis=1, inplace=True)

# Check the feature importances
importances = pd.DataFrame({
    'Feature': train_data_transformed.columns,
    'Importance': models_lgb[0].feature_importances_
}).sort_values(by='Importance', ascending=False)
importances


In [ ]:
# Calculate the 75th percentile of the feature importance
threshold = importances['Importance'].quantile(0.75)

# Select features whose importance is higher than the threshold
selected_features = importances[importances['Importance'] > threshold]['Feature'].tolist()

selected_features

In [ ]:
preds = []
for transformed_test_data, model in zip(transformed_test_data_list, models_lgb):
    pred = model.predict(transformed_test_data)
    preds.append(pred)
pred = np.mean(preds, axis=0)

# submission
sub_df["price"] = pred
sub_df.to_csv(f'{model_dir}/submission.csv', index=False, header=False)


In [ ]:
visualize_importance(models, train_feat_df)
visualize_oof_gt(oof, train_df['price'])
visualize_oof_pred(oof, pred)